## LFM降维方法-矩阵因子分解

In [1]:
### 0.引入依赖
import numpy as np
import pandas as pd

In [2]:
### 1.准备数据
# 评分矩阵R
R = np.array([[4,0,2,0,1],
             [0,2,3,0,0],
             [1,0,2,4,0],
             [5,0,0,3,1],
             [0,0,1,5,1],
             [0,3,2,4,1],])
len(R[0])

5

In [3]:
### 2.核心算法实现
"""
@输入参数：
R：M*N 的评分矩阵
K：隐特征向量维度
max_iter: 最大迭代次数
alpha：步长
lamda：正则化系数

@输出：
分解之后的 P，Q
P：初始化用户特征矩阵M*K
Q：初始化物品特征矩阵N*K
"""

# 给定超参数

K = 5
max_iter = 5000
alpha = 0.0002
lamda = 0.004

# 核心算法
def LFM_grad_desc( R, K=2, max_iter=1000, alpha=0.0001, lamda=0.002 ):
    # 基本维度参数定义
    M = len(R)
    N = len(R[0])
    
    # P,Q初始值，随机生成
    P = np.random.rand(M, K)
    Q = np.random.rand(N, K)
    Q = Q.T
    
    # 开始迭代
    for step in range(max_iter):
        # 对所有的用户u、物品i做遍历，对应的特征向量Pu、Qi梯度下降
        for u in range(M):
            for i in range(N):
                # 对于每一个大于0的评分，求出预测评分误差
                if R[u][i] > 0:
                    eui = np.dot( P[u,:], Q[:,i] ) - R[u][i]
                    
                    # 代入公式，按照梯度下降算法更新当前的Pu、Qi
                    for k in range(K):
                        P[u][k] = P[u][k] - alpha * ( 2 * eui * Q[k][i] + 2 * lamda * P[u][k] )
                        Q[k][i] = Q[k][i] - alpha * ( 2 * eui * P[u][k] + 2 * lamda * Q[k][i] )
        
        # u、i遍历完成，所有特征向量更新完成，可以得到P、Q，可以计算预测评分矩阵
        predR = np.dot( P, Q )
        
        # 计算当前损失函数
        cost = 0
        for u in range(M):
            for i in range(N):
                if R[u][i] > 0:
                    cost += ( np.dot( P[u,:], Q[:,i] ) - R[u][i] ) ** 2
                    # 加上正则化项
                    for k in range(K):
                        cost += lamda * ( P[u][k] ** 2 + Q[k][i] ** 2 )
        if cost < 0.0001:
            break
        
    return P, Q.T, cost

In [4]:
### 3.测试
P, Q, cost = LFM_grad_desc(R, K, max_iter, alpha, lamda)

print(P)
print(Q)
print(cost)

predR = P.dot(Q.T)

print(R)
predR

[[ 3.45342034e-01  4.74623440e-01  9.26460029e-01  9.66959136e-01
   1.01772195e+00]
 [ 4.05200045e-01  1.57468836e+00  3.30657156e-01  5.94536950e-01
   6.54199735e-01]
 [ 8.00096373e-01  1.22447529e+00  2.10807396e-02 -9.64849744e-02
   8.04188906e-01]
 [ 1.41904456e-02  6.39575712e-02  1.43875891e+00  1.28614660e+00
   8.08665322e-01]
 [ 1.13387594e+00  4.26209104e-01  1.20863675e+00 -1.48448771e-03
   1.01387146e+00]
 [ 6.95073010e-01  5.57042595e-01  3.97515493e-01  8.43714642e-01
   1.23080235e+00]]
[[ 0.27984559  0.22203325  1.6053379   1.57548251  0.77794761]
 [ 0.82855169  0.25161287  0.29416755  0.55092545  1.34024132]
 [ 0.02536607  1.38188037  0.02064222  0.85189532  0.41974258]
 [ 1.60474166  1.47216318  1.13052066  0.2618756   1.14155886]
 [ 0.26629887  0.68366127  0.33440983  0.39005465 -0.04099233]]
0.5541313816526678
[[4 0 2 0 1]
 [0 2 3 0 0]
 [1 0 2 4 0]
 [5 0 0 3 1]
 [0 0 1 5 1]
 [0 3 2 4 1]]


array([[4.00446759, 2.57480497, 1.93468618, 3.71530262, 1.06171132],
       [2.43945877, 2.0335407 , 2.97421349, 4.24475658, 1.5001175 ],
       [1.00322582, 2.00186739, 1.96816604, 4.00316953, 0.98664071],
       [4.98326677, 2.24346382, 1.55353439, 3.00342412, 0.99715773],
       [3.13861324, 2.76027082, 1.06698131, 4.97042132, 0.95537257],
       [3.24309918, 2.94739519, 2.03097981, 4.01085129, 0.97750006]])